In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

from spacy.tokens import DocBin
import srsly
import spacy
import spacy
from spacy.training import offsets_to_biluo_tags

pd.set_option('display.max_colwidth', None)

# Prep

In [2]:
import os
os.chdir('/Users/anthony/Projects/retailer_nlp_challenger/data')

# read csv
brand_cleaned = pd.read_csv('brand_offer_cleaned.csv')
brand_cleaned.sample(10)

,BRAND,OFFER,idx
280,KFC,Spend 10 at KFC,379
205,KFC,Spend 40 at KFC,289
112,BRITA,Brita Pitcher AND Filter,158
12,RED GOLD,Red Gold Tomato Ketchup,18
39,SHAWS,Spend 185 at Shaws,53
105,WELCHS FRUIT SNACKS,"Welchs Fruit Snacks, 250 count, online at Amazon",147
76,PERFECT KETO,"Perfect Keto Beauty Sleep Collagen, online at Amazon",112
60,MARINELA,"Marinela sweet baked goods, buy 2",85
218,BROWNIE BRITTLE,"Brownie Brittle Snacks, select varieties, buy 2",304
209,SARGENTO,"Sargento Sliced Cheese, spend 18",294


In [3]:
# read csv
retailer_cleaned = pd.read_csv('retailer_offer_cleaned.csv')
retailer_cleaned.sample(10)

,RETAILER,OFFER,idx
149,WALMART,"Arber, at Walmart",325
77,STAR MARKET,Shop 2 times at Star Market,170
44,WALMART,"Back to the Roots Soils, select varieties and sizes, at Walmart",86
14,ZAXBYS,Order online at Zaxbys.com,30
42,SUBWAY,Spend 15 at Subway,82
123,TARGET,"Arber, at Target",270
177,WALMART,"Back to the Roots Soils, select varieties, at Walmart",383
74,AMAZON,"Perfect Keto MCT Oil Powder, online at Amazon",156
25,VONS,Any Vons receipt,50
65,LOWES HOME IMPROVEMENT,"Back to the Roots Garden Soil, 1 cubic foot, at Lowes Home Improvement",137


In [4]:
# use brand_cleaned to left join retailer_cleaned (cols: idx and retailer)
merged = pd.merge(brand_cleaned, retailer_cleaned[['idx','RETAILER']], on='idx', how='left')
merged.sample(10)

,BRAND,OFFER,idx,RETAILER
260,STAR MARKET,Any Star Market receipt,354,STAR MARKET
145,CORE HYDRATION,"CORE Hydration, select varieties, at Walmart",205,WALMART
254,CVS,Spend 30 at CVS,347,CVS
252,NATURE VALLEY,Nature Valley Protein Granola,345,NaN
283,PAVILIONS,Spend 270 at Pavilions,382,PAVILIONS
191,ARBER,"Arber, at Target",270,TARGET
152,KLONDIKE,"Klondike Cones, spend 10 at Walmart",216,WALMART
276,SUPER COFFEE,"Super Coffee, 12 ounce single serve",374,NaN
12,RED GOLD,Red Gold Tomato Ketchup,18,NaN
6,EGGLIFE,Egglife Egg White Wraps at Aldi,10,ALDI


# Brand

## Train-test-val split

In [5]:
df_merge = merged.copy()
df_merge['RETAILER'] = df_merge['RETAILER'].fillna('').astype(str)

In [6]:
# split data into train, test, and validation (80%, 10%, 10%)
train = df_merge.sample(frac=0.8, random_state=42)
dev = df_merge.drop(train.index)
val = train.sample(frac=0.05, random_state=42)

# val is seperated from train, so it should be dropped from train
# rows in train that has the same idx as val (the feature, not the index) should be dropped
train = train[~train['idx'].isin(val['idx'])]

In [7]:
print(f'train size: {train.shape}')
print(f'dev size: {dev.shape}')
print(f'val size: {val.shape}')

train size: (217, 4)
dev size: (57, 4)
val size: (11, 4)


## Convert csv to json

Sometimes you have:
1. Retailer: ABC, OFFER: save $10 at ABC.com
2. Retailer: ABC.com, OFFER: save $10 at ABC
3.  Retailer: ABC.com, OFFER: save $10 at ABC.com
4.  Retailer: ABC, OFFER: save $10 at ABC

3 & 4 easy to fix. How do we fix 1 & 2? We can use EntityRuler.

Another headache is that some retailers also have their same-name brands. For example, Sam's Club. When a customer only searches "Sam's Club", we won't be able to know whether it is a brand or a retailer. This leads to the problem of span labeling. That is, `('Spend 15 at Burger King', [[12, 23, 'BRAND'], [12, 23, 'RETAILER'])` is labeling the same entity at the same time. This is a problem of ambiguity, which is making this task more complex and might be better to solve it in the similarity calculation stage. I am making an assumption here that these retailer-owned brands should be annotated as `RETAILER` during labeling.

Based on the EDA notebook, we know that the retailer-owned-brand is only sold by the retailers themselves. Therefore, when a customer search "Sam's Club", we

In [52]:
# Generate a list of [start_char,end_char, label] of given label, tagname is the name of the label
# Spacy example: 
# ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")])

def label_tagger(text, label, tag):
    """Takes in a string, returns the tuple of (start_char, end_char, label)"""
    text = text.lower()
    label = label.lower().strip()
    if text.find(label) != -1:
        start_index = text.find(label)
        last_index = start_index + len(label)
        return [start_index,last_index,tag]
    else:
        print(text)
        print(f"{label} not found in the given sentence")

def df_labeller(df):
    """Convert dataframe to a list of tuples"""
    ner_output = []
    for i,row in enumerate(df.itertuples(),1):
        offer, brand, retailer = row.OFFER, row.BRAND, row.RETAILER
        brand_label = label_tagger(offer, brand, "BRAND")
        if (len(retailer) > 0) and (retailer != brand): # we filled NaN with empty string, need to filter out 
            retailer_label = label_tagger(offer, retailer, "RETAILER") 
            ner_datapoint = (offer, [brand_label, retailer_label])
        else:
            ner_datapoint = (offer, [brand_label])
        ner_output.append(ner_datapoint)
    return ner_output

In [53]:
# data convert
train_labeled = df_labeller(train)
dev_labeled = df_labeller(dev)
val_labeled = df_labeller(val)

In [10]:
train_labeled[:10]

[('DOVE Chocolate, select sizes, buy 1', [[0, 14, 'BRAND']]),
 ('The Rustik Oven bread', [[4, 15, 'BRAND']]),
 ('When you join Costco as an Executive Member New Members Only',
  [[14, 20, 'BRAND']]),
 ('Sargento Sliced Cheese, spend 18', [[0, 8, 'BRAND']]),
 ('Sargento Sliced Cheese, spend 12', [[0, 8, 'BRAND']]),
 ('Barilla pasta, select varieties, buy 4', [[0, 7, 'BRAND']]),
 ('Spend 15 at Burger King', [[12, 23, 'BRAND']]),
 ('Gortons at select retailers', [[0, 7, 'BRAND']]),
 ('Sara Lee bread, select varieties, buy 2 at Walmart',
  [[0, 8, 'BRAND'], [43, 50, 'RETAILER']]),
 ('Any Albertsons receipt', [[4, 14, 'BRAND']])]

In [65]:
# save data
ents_label = {}
ents_label['train'] = train_labeled
ents_label['dev'] = dev_labeled
ents_label['val'] = val_labeled

with open('ents_label.json', 'w') as fp:
    json.dump(ents_label, fp)

# also save val for later use
val.to_csv('val.csv',index=False)

In [ ]:
# also write them into csv
# train.to_csv('train.csv',index=False)
# dev.to_csv('test.csv',index=False)
# val.to_csv('val.csv',index=False)

## JSON to spaCy binary

In [66]:
# read json
with open('ents_label.json') as json_file:
    ents_label_dict = json.load(json_file)

train_input = ents_label_dict['train']
dev_input = ents_label_dict['dev']
val_input = ents_label_dict['val']

In [ ]:
convert("en","train.json","/Users/anthony/Projects/retailer_nlp_challenger/corpus/train.spacy")

In [ ]:
convert("en","dev.json","/Users/anthony/Projects/retailer_nlp_challenger/corpus/dev.spacy")

In [73]:
import spacy
from spacy.tokens import DocBin

# test = df_labeller(train[train.RETAILER.eq("COSTCO")])
# test = train_input[:20]

def conver_data_to_spacy(data, to_path, lang='en'):
    nlp = spacy.blank(lang)
    db = DocBin()
    skipped = 0
    for text, annotations in data:
        doc = nlp(text)
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            if span is None:
                skipped += 1
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                print(msg)
            else:
                # Check if the span overlaps with any existing entity spans
                overlaps = [ent for ent in ents if ent.start < span.end and ent.end > span.start]
                if overlaps:
                    # Remove any overlapping entity spans
                    for ent in overlaps:
                        ents.remove(ent)
                        msg = f"Removed overlapping entity [{ent.start}, {ent.end}, {ent.label_}] in the following text:\n\n{repr(text)}\n"
                        print(msg)
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    print('### Totally skipped docs:', skipped)
    db.to_disk(to_path)

In [74]:
path = "/Users/anthony/Projects/retailer_nlp_challenger/corpus/"
conver_data_to_spacy(train_input, path+"train.spacy")

Skipping entity [0, 5, BRAND] in the following text because the character span 'Pepsi' does not align with token boundaries:

'PepsiCo Variety Pack, select varieties, at Amazon Storefront'

Skipping entity [16, 22, BRAND] in the following text because the character span 'Zaxbys' does not align with token boundaries:

'Order online at Zaxbys.com'

Skipping entity [0, 5, BRAND] in the following text because the character span 'Pepsi' does not align with token boundaries:

'PepsiCo Beverage, 7.5-ounce 10 pack, select varieties, at Amazon Storefront'

Skipping entity [12, 17, BRAND] in the following text because the character span 'Chewy' does not align with token boundaries:

'Spend 50 at Chewy.com'

Skipping entity [35, 40, RETAILER] in the following text because the character span 'Chewy' does not align with token boundaries:

'Kradle, select products, online at Chewy.com'

Skipping entity [0, 5, BRAND] in the following text because the character span 'Aleve' does not align with token b

In [75]:
path = "/Users/anthony/Projects/retailer_nlp_challenger/corpus/"
conver_data_to_spacy(dev_input, path+"train.spacy")

Removed overlapping entity [3, 6, BRAND] in the following text:

'Spend 35 at Fresh Thyme Market'

### Totally skipped docs: 0
